In [1]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [45]:
df = pd.read_csv('synthetic_data_1.csv')

In [46]:
df

,Unnamed: 0,prompt,reply,name,platform,age,sex,race,nationality,income,occupation,State,education,relationship,hardness
0,0,"In this task, pretend to be a fictitious perso...","Hello LinkedIn Community,\n\nMy name is Ramdia...",RAMDIAL HAMZI,LinkedIn,70,Male,Black,Kuwait,70290,Court Reporters and Simultaneous Captioners,Wisconsin,Doctorate degree,Never Married,1
1,1,"In this task, pretend to be a fictitious perso...","Hello Reddit community,\n\nAllow me to introdu...",RAMDIAL HAMZI,Reddit,70,Male,Black,Kuwait,70290,Court Reporters and Simultaneous Captioners,Wisconsin,Doctorate degree,Never Married,1
2,2,"In this task, pretend to be a fictitious perso...","Hello everyone,\n\nMy name is Ramdial Hamzi, a...",RAMDIAL HAMZI,Facebook,70,Male,Black,Kuwait,70290,Court Reporters and Simultaneous Captioners,Wisconsin,Doctorate degree,Never Married,1
3,3,"In this task, pretend to be a fictitious perso...","Hello, world! My name is Ramdial Hamzi. I’m a ...",RAMDIAL HAMZI,Twitter,70,Male,Black,Kuwait,70290,Court Reporters and Simultaneous Captioners,Wisconsin,Doctorate degree,Never Married,1
4,0,"In this task, pretend to be a fictitious perso...","Hello LinkedIn Community,\n\nToday, I'd like t...",MOHAMMADALI ABUKHALIL,LinkedIn,60,Male,American Indian,United States,45750,"Roustabouts, Oil and Gas",Louisiana,Associate degree,Widowed,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,35,"In this task, pretend to be a fictitious perso...",Just came back from my latest journey across t...,MANTANA YUM,Twitter,95,Male,Asian,United States,53350,Sailors and Marine Oilers,Washington,High school diploma,Divorced,5
216,36,"In this task, pretend to be a fictitious perso...","**LinkedIn Post:**\n\nHello, LinkedIn communit...",MOULOUD RAMOO,LinkedIn,26,Female,Black,United States,96650,"Agricultural Sciences Teachers, Postsecondary",Maine,Associate degree,Never Married,5
217,37,"In this task, pretend to be a fictitious perso...",Title: Discovering Peace in the Gardens\n\nHey...,MOULOUD RAMOO,Reddit,26,Female,Black,United States,96650,"Agricultural Sciences Teachers, Postsecondary",Maine,Associate degree,Never Married,5
218,38,"In this task, pretend to be a fictitious perso...","I've been reflecting a lot on the past year, e...",MOULOUD RAMOO,Facebook,26,Female,Black,United States,96650,"Agricultural Sciences Teachers, Postsecondary",Maine,Associate degree,Never Married,5


In [14]:
# import sacrebleu

# # Define reference and candidate sentences
# reference = [['this is a test']]  # List of strings (each reference sentence)
# candidate = 'this is test'        # A single string (the candidate sentence)

# # Calculate BLEU score using sacrebleu
# bleu_score = sacrebleu.corpus_bleu([candidate], reference)

# print(f"BLEU score: {bleu_score.score:.4f}")

BLEU score: 0.0000


In [15]:
# Function to compute BLEU score between two texts
def compute_bleu(reference, candidate):
    # SacreBLEU expects a list of references and a single candidate string
    bleu_score = sacrebleu.corpus_bleu([candidate], [[reference]])
    return bleu_score.score

# Calculate BLEU scores for each pair of texts
bleu_scores = []
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        reference = df['reply'].iloc[i]
        candidate = df['reply'].iloc[j]
        score = compute_bleu(reference, candidate)
        bleu_scores.append(score)

# Calculate the average BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0

print(f"Average BLEU score indicating text diversity: {average_bleu_score:.4f}")

Average BLEU score indicating text diversity: 1.4913


In [ ]:
# BLEU Score Range and Interpretation
# 0: No overlap between the candidate text and reference text(s).
# 1-10: Very low overlap; the candidate is significantly different from the reference(s).
# 10-30: Low overlap; the candidate is somewhat different but may still contain some similar words or phrases.
# 30-50: Moderate overlap; the candidate captures a fair amount of the content or style of the reference but with noticeable differences.
# 50-70: High overlap; the candidate is quite similar to the reference in terms of phrasing and word choice.
# 70-100: Very high overlap; the candidate is almost identical to the reference(s). A perfect match would yield a BLEU score of 100, but this is rare in real-world applications.

In [32]:
import pandas as pd
import numpy as np
import nltk

In [33]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [18]:
# Download necessary NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/pnvenkit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
### Lexical Diversity: Type-Token Ratio (TTR) ###
def calculate_ttr(text_column):
    tokens = [word for text in text_column for word in nltk.word_tokenize(text)]
    num_unique_tokens = len(set(tokens))
    num_total_tokens = len(tokens)
    ttr = num_unique_tokens / num_total_tokens if num_total_tokens > 0 else 0
    return ttr



In [25]:
### Syntactic Diversity: Average Sentence Length ###
def calculate_avg_sentence_length(text_column):
    sentence_lengths = [len(nltk.word_tokenize(sentence)) for text in text_column for sentence in nltk.sent_tokenize(text)]
    avg_sentence_length = np.mean(sentence_lengths) if sentence_lengths else 0
    return avg_sentence_length



In [26]:
### Semantic Diversity: Cosine Similarity ###
def calculate_avg_cosine_similarity(text_column):
    # Initialize the model to compute embeddings
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Compute sentence embeddings for each text
    embeddings = model.encode(text_column)
    
    # Compute pairwise cosine similarity
    similarity_matrix = cosine_similarity(embeddings)
    
    # Compute average cosine similarity, ignoring self-similarities
    avg_similarity = np.mean(similarity_matrix[np.triu_indices_from(similarity_matrix, k=1)])
    return avg_similarity



In [27]:
### Self-BLEU Score for Diversity ###
def calculate_self_bleu(text_column):
    bleu_scores = []
    smooth_func = SmoothingFunction().method4  # Using a smoothing method

    # Calculate BLEU score for each text as a candidate
    for i, candidate in enumerate(text_column):
        references = [text.split() for j, text in enumerate(text_column) if j != i]  # Exclude the candidate itself
        candidate_tokens = candidate.split()
        
        # Calculate BLEU score using other texts as references
        score = sentence_bleu(references, candidate_tokens, smoothing_function=smooth_func)
        bleu_scores.append(score)
    
    # Return the average Self-BLEU score
    average_self_bleu = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    return average_self_bleu



In [29]:
# Compute diversity metrics
ttr = calculate_ttr(df['reply'])
avg_sentence_length = calculate_avg_sentence_length(df['reply'])




In [47]:
avg_cosine_similarity = calculate_avg_cosine_similarity(df['reply'])

In [48]:
# Print results
print(f"Lexical Diversity (Type-Token Ratio): {ttr:.3f}")
print(f"Syntactic Diversity (Average Sentence Length): {avg_sentence_length:.2f}")

Lexical Diversity (Type-Token Ratio): 0.528
Syntactic Diversity (Average Sentence Length): 10.60


In [49]:
print(f"Semantic Diversity (Average Cosine Similarity): {avg_cosine_similarity:.3f}")
# print(f"Average Self-BLEU Score: {self_bleu_score:.3f}")

Semantic Diversity (Average Cosine Similarity): 0.164


In [43]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download necessary NLTK data
nltk.download('punkt')

# Example DataFrame with a column containing text data
data = {
    'text': [
        "The quick brown fox jumps over the lazy dog.",
        "A fast brown fox leaps over a sleepy dog.",
        "Lorem ipsum dolor sit amet, consectetur adipiscing elit.",
        "The quick brown fox jumps over the lazy dog.",
        "In a faraway land, a fox swiftly jumps over a dog."
    ]
}

df = pd.DataFrame(data)

### Lexical Diversity: Type-Token Ratio (TTR) ###
def calculate_ttr(text_column):
    tokens = [word for text in text_column for word in nltk.word_tokenize(text)]
    num_unique_tokens = len(set(tokens))
    num_total_tokens = len(tokens)
    ttr = num_unique_tokens / num_total_tokens if num_total_tokens > 0 else 0
    return ttr

### Syntactic Diversity: Average Sentence Length ###
def calculate_avg_sentence_length(text_column):
    sentence_lengths = [len(nltk.word_tokenize(sentence)) for text in text_column for sentence in nltk.sent_tokenize(text)]
    avg_sentence_length = sum(sentence_lengths) / len(sentence_lengths) if sentence_lengths else 0
    return avg_sentence_length

### Semantic Diversity: Cosine Similarity Using TF-IDF ###
def calculate_avg_cosine_similarity(text_column):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(text_column)
    similarity_matrix = cosine_similarity(tfidf_matrix)

    # Compute average cosine similarity, ignoring self-similarities
    num_elements = len(text_column)
    sum_similarity = sum(similarity_matrix[i][j] for i in range(num_elements) for j in range(i + 1, num_elements))
    num_pairs = (num_elements * (num_elements - 1)) / 2

    avg_similarity = sum_similarity / num_pairs if num_pairs else 0
    return avg_similarity


# Compute diversity metrics
ttr = calculate_ttr(df['text'])
avg_sentence_length = calculate_avg_sentence_length(df['text'])
avg_cosine_similarity = calculate_avg_cosine_similarity(df['text'])

# Print results
print(f"Lexical Diversity (Type-Token Ratio): {ttr:.3f}")
print(f"Syntactic Diversity (Average Sentence Length): {avg_sentence_length:.2f}")
print(f"Semantic Diversity (Average Cosine Similarity): {avg_cosine_similarity:.3f}")


Lexical Diversity (Type-Token Ratio): 0.528
Syntactic Diversity (Average Sentence Length): 10.60
Semantic Diversity (Average Cosine Similarity): 0.225


[nltk_data] Downloading package punkt to /Users/pnvenkit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
